#### Setup

In [ ]:
%rebootdevice

In [ ]:
%disconnect

#### Connect board (wlan)

In [ ]:
%websocketconnect --raw   ws://192.168.0.202:8266 --password pass

#### Connect board

In [3]:
%serialconnect --port /dev/cu.usbserial-02215408

Connecting to --port=/dev/cu.usbserial-02215408 --baud=115200 
Ready.


#### Setup watch

In [3]:
from machine import Pin, SPI # type: ignore
import axp202c
import st7789

TFA = 0
BFA = 80

# Enable Power management
axp = axp202c.PMU()
axp.enablePower(axp202c.AXP202_LDO2)

tft = st7789.ST7789(
    SPI(1, baudrate=32000000, sck=Pin(18, Pin.OUT), mosi=Pin(19, Pin.OUT)),
    240,
    240,
    cs=Pin(5, Pin.OUT),
    dc=Pin(27, Pin.OUT),
    backlight=Pin(12, Pin.OUT),
    rotation=2,
    options=0,
    buffer_size=0)
tft.init()

* initializing pins
* initializing i2c
* initializing mpu
* Detect PMU Type is AXP202


#### Power management

In [ ]:
import sys
import time

tft.off()
prev_power = 0
while True:
    curr_power = axp.getBattPercentage()
    if curr_power != prev_power:
        print(f"Power: {curr_power}")
    time.sleep(3)
    prev_power = curr_power

#### Radar ui

In [ ]:
import time
import vga1_16x16 as font
from machine import I2C, Pin
from math import sqrt
from bma423 import BMA423

# Clear screen
tft.fill(st7789.BLACK)

# 1. Make Circle
tft.circle(120, 120, 100, st7789.GREEN)
tft.circle(120, 120, 50, st7789.YELLOW)

# 2. Add X, Y line
tft.hline(0, 120, 240, st7789.BLUE)
tft.vline(120, 0, 240, st7789.BLUE)
tft.text(font, "X", 220, 130, st7789.RED)
tft.text(font, "Y", 130, 10, st7789.RED)

# 3. Create arrow line to point to original location


# 4. BMA location coordinations in X, Y
# Initialize I2C (T-Watch default pins: scl=22, sda=21)
i2c = I2C(1, scl=Pin(22), sda=Pin(21))
# Initialize BMA423
sensor = BMA423(i2c)

# Variables for velocity and displacement
vx, vy, vz = 0, 0, 0
x_pos, y_pos, z_pos = 0, 0, 0
prev_time = time.ticks_ms()
threshold = 0.1  # Define a threshold for movement detection

# Main loop to calculate movement
while True:
    # Get current time
    current_time = time.ticks_ms()
    dt = time.ticks_diff(current_time, prev_time) / 1000  # Convert to seconds

    # Read accelerometer data (in mg)
    ax, ay, az = sensor.get_xyz()

    # Convert from mg to cm/s²
    ax= (ax * 981) / 1000
    ay = (ay * 981) / 1000
    az = (az * 981) / 1000

    print(f"Accelaration: X={ax:.2f}, Y={ay:.2f}, Z={az:.2f} (m/s2)")

    # Apply threshold to reduce noise
    if abs(ax) < threshold:
        ax = 0
    if abs(ay) < threshold:
        ay = 0
    if abs(az) < threshold:
        az = 0

    # Total acceleration (magnitude)
    total_accel = sqrt(ax**2 + ay**2 + az**2)
    print(f"Total Acceleration: {total_accel:.2f} m/s^2")

    # # If total acceleration is below the threshold, reset velocities
    # if total_accel < 0.1:
    #     vx, vy, vz = 0, 0, 0
    # else:
    # Estimate velocity (v = v0 + a * dt)
    vx += ax * dt
    vy += ay * dt
    vz += az * dt

    # Estimate displacement (x = x0 + v * dt)
    x_pos += vx * dt
    y_pos += vy * dt
    z_pos += vz * dt

    print(f"Velocity: X={vx:.2f}, Y={vy:.2f}, Z={vz:.2f} (m/s)")
    print(f"Position: X={x_pos:.2f}, Y={y_pos:.2f}, Z={z_pos:.2f} (m)")

    # Update previous time
    prev_time = current_time

    time.sleep(0.1)  # Sleep for a short duration

#### Test display

In [4]:
import random
import time

# Clear screen
tft.fill(st7789.BLACK)

# Fill screen random
for x in range(0,240):
    # time.sleep_us(1)
    for y in range(0, 240):
        color = st7789.color565(
                        random.getrandbits(8),
                        random.getrandbits(8),
                        random.getrandbits(8))
        tft.pixel(x, y, color)

.....